In [2]:
tps = []
N = []
PE = []
timesteps = []
Ns = [100,200,300,400,500]
densities = [0.01, 0.005, 0.001, 0.001, 0.0005]
dts = []

In [3]:
import flowermd
import gsd
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import numpy as np
import warnings
from cmeutils.visualize import FresnelGSD
from flowermd.base import Pack, Simulation, System, Molecule
from flowermd.base.forcefield import BaseHOOMDForcefield
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.library.forcefields import BeadSpring
from flowermd.utils import get_target_box_number_density
from mbuild.compound import Compound
from mbuild.lattice import Lattice
import unyt as u
warnings.filterwarnings('ignore')
dt = 0.0005
N_chains = 100
initial_dens = 0.001
final_dens = 0.3
cpu = hoomd.device.CPU()
class Graphene(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        )
        Flakium = Compound(name="F", element="F") # defines a carbon atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined carbon atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated carbon lattice layers to the 'surface' compound, which represents our graphene structure
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Graphene, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]
#OK, so we want to initialize a system with some chains and some flakes
kg_chain = LJChain(lengths=10,num_mols=N_chains)
sheet = Graphene(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False))
system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=10), kg_chain],
              density=initial_dens, packing_expand_factor = 6, seed=2)
system.visualize()

/Users/joselyngutierrez/miniconda3/envs/flowermd-dev/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
target_box = get_target_box_number_density(density=final_dens*u.Unit("nm**-3"),n_beads=(500+(N_chains*10)))

In [5]:
# this FF is for research question
# WCA = 2.5 **1/6, kT = 3.0, needs thousand chains, maybe 10-12 flakes. large system, need lots of steps. 5e6 probably good.
ff = BeadSpring(
    r_cut=2**(1/6),
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # increased k to avoid chain collapse
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   # moderate stiffness for chains
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), #need to turn this on later, messed up with straight chains
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)
gsd = f"{N_chains}_10mer10f_{dt}dt_7kT_CPUtest.gsd"
log = f"{N_chains}_10mer10f_{dt}dt_7kT_CPUtest.txt"
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=cpu, dt = dt, gsd_write_freq=int(1000), log_file_name = log, gsd_file_name = gsd)
sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=5e6,tau_kt=100*sim.dt,period=10,thermalize_particles=True)
sim.run_NVT(n_steps=1e6, kT=7, tau_kt=dt*100)
sim.flush_writers()

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 5000000; TPS: 1241.87; ETA: 1.0 hours, 7.0 minutes
Step 2000 of 5000000; TPS: 1754.86; ETA: 47.5 minutes
Step 3000 of 5000000; TPS: 2026.99; ETA: 41.1 minutes
Step 4000 of 5000000; TPS: 2200.38; ETA: 37.8 minutes
Step 5000 of 5000000; TPS: 2317.37; ETA: 35.9 minutes
Step 6000 of 5000000; TPS: 2403.4; ETA: 34.6 minutes
Step 7000 of 5000000; TPS: 2470.62; ETA: 33.7 minutes
Step 8000 of 5000000; TPS: 2399.15; ETA: 34.7 minutes
Step 9000 of 5000000; TPS: 2450.05; ETA: 34.0 minutes
Step 10000 of 5000000; TPS: 2492.68; ETA: 33.4 minutes
Step 11000 of 5000000; TPS: 2440.42; ETA: 34.1 minutes
Step 12000 of 5000000; TPS: 2476.83; ETA: 33.6 minutes
Step 13000 of 5000000; TPS: 2501.97; ETA: 33.2 minutes
Step 14000 of 5000000; TPS: 2531.34; ETA: 32.8 minutes
Step 15000 of 5000000; TPS: 2555.56; ETA: 32.5 minutes
Step 16000 of 5000000; TPS: 2578.73; ETA: 32.2 minutes
Step 17000 of 5000000; TPS: 2597.56; ETA: 32.0 minutes
Step 18000 

In [6]:
data = np.genfromtxt(log, names=True)

In [7]:
N.append(N_chains)

In [8]:
PE.append(data["mdcomputeThermodynamicQuantitiespotential_energy"])

In [9]:
timesteps.append(data["flowermdbasesimulationSimulationtimestep"])

In [10]:
densities.append(final_dens)

In [11]:
dts.append(dt)

In [12]:
np.savetxt('N.txt', N, delimiter=' ')
np.savetxt('tps.txt', tps, delimiter=' ')
np.savetxt('PE.txt', PE, delimiter=' ')
np.savetxt('timesteps.txt', timesteps, delimiter=' ')
np.savetxt('densities.txt', densities, delimiter=' ')
np.savetxt('dts.txt', dts, delimiter=' ')